# APS Experimental Results Analysis

This notebook provides interactive analysis of all experimental results from Phases 5.2-5.5.

**Contents:**
1. Load and explore all results
2. Ablation study visualization
3. OOD robustness analysis
4. Few-shot learning evaluation
5. Component contribution analysis
6. Statistical significance tests

In [ ]:
import json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

%matplotlib inline

## 1. Load All Results

In [ ]:
# Load ablation study results
ablation_path = Path("../outputs/ablation/ablation_summary.json")
with open(ablation_path, "r") as f:
    ablation_results = json.load(f)

print("Loaded ablation results for configurations:")
print(list(ablation_results.keys()))

In [ ]:
# Load OOD results
ood_dir = Path("../outputs/ood/metrics")
ood_results = {}
for json_file in ood_dir.glob("*_ood_results.json"):
    config_name = json_file.stem.replace("_ood_results", "")
    with open(json_file, "r") as f:
        ood_results[config_name] = json.load(f)

print(f"Loaded OOD results for {len(ood_results)} configurations")

In [ ]:
# Load few-shot results
fewshot_dir = Path("../outputs/fewshot/metrics")
fewshot_results = {}
for json_file in fewshot_dir.glob("*_fewshot_results.json"):
    config_name = json_file.stem.replace("_fewshot_results", "")
    with open(json_file, "r") as f:
        fewshot_results[config_name] = json.load(f)

print(f"Loaded few-shot results for {len(fewshot_results)} configurations")

## 2. Ablation Study Analysis

In [ ]:
# Create comparison dataframe
configs = ["baseline", "t_only", "c_only", "e_only", "t_c", "t_e", "c_e", "t_c_e"]
metrics_data = []

for config in configs:
    if config in ablation_results:
        metrics = ablation_results[config]["metrics"]
        metrics_data.append({
            "Config": config,
            "Reconstruction": metrics["reconstruction_error"],
            "Trustworthiness": metrics["trustworthiness"],
            "Continuity": metrics["continuity"],
            "kNN Preservation": metrics["knn_preservation"],
            "ARI": metrics["ari"],
            "NMI": metrics["nmi"],
            "Silhouette": metrics["silhouette"],
        })

df_ablation = pd.DataFrame(metrics_data)
df_ablation

In [ ]:
# Visualize topology metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ["Trustworthiness", "Continuity", "kNN Preservation"]
for ax, metric in zip(axes, metrics):
    df_ablation.plot(x="Config", y=metric, kind="bar", ax=ax, legend=False)
    ax.set_title(metric)
    ax.set_xlabel("Configuration")
    ax.set_ylabel("Score")
    ax.set_xticklabels(df_ablation["Config"], rotation=45)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize clustering metrics
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

metrics = ["ARI", "NMI", "Silhouette"]
for ax, metric in zip(axes, metrics):
    df_ablation.plot(x="Config", y=metric, kind="bar", ax=ax, legend=False, color="coral")
    ax.set_title(metric)
    ax.set_xlabel("Configuration")
    ax.set_ylabel("Score")
    ax.set_xticklabels(df_ablation["Config"], rotation=45)
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Component Contribution Analysis

Analyze how each component (T, C, E) contributes to overall performance.

In [ ]:
# Calculate improvements over baseline
baseline_metrics = ablation_results["baseline"]["metrics"]

improvements = []
for config in ["t_only", "c_only", "e_only", "t_c", "t_e", "c_e", "t_c_e"]:
    if config in ablation_results:
        metrics = ablation_results[config]["metrics"]
        improvements.append({
            "Config": config,
            "Trust Δ%": ((metrics["trustworthiness"] - baseline_metrics["trustworthiness"]) / baseline_metrics["trustworthiness"]) * 100,
            "Cont Δ%": ((metrics["continuity"] - baseline_metrics["continuity"]) / baseline_metrics["continuity"]) * 100,
            "ARI Δ%": ((metrics["ari"] - baseline_metrics["ari"]) / baseline_metrics["ari"]) * 100,
        })

df_improvements = pd.DataFrame(improvements)
df_improvements

In [ ]:
# Heatmap of improvements
plt.figure(figsize=(10, 6))
sns.heatmap(
    df_improvements.set_index("Config"),
    annot=True,
    fmt=".2f",
    cmap="RdYlGn",
    center=0,
    cbar_kws={'label': 'Improvement (%)'}
)
plt.title("Improvement over Baseline (%)")
plt.tight_layout()
plt.show()

## 4. OOD Robustness Analysis

In [ ]:
# Compare baseline vs t_c_e on rotation
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Rotation robustness
ax = axes[0]
for config in ["baseline", "t_c_e"]:
    if config in ood_results:
        angles = [15, 30, 45, 60]
        accs = [ood_results[config]["rotation"][f"rot_{a}"]["knn_accuracy"] for a in angles]
        ax.plot(angles, accs, marker='o', linewidth=2, label=config)

ax.set_xlabel("Rotation Angle (degrees)")
ax.set_ylabel("kNN Accuracy")
ax.set_title("Rotation Robustness")
ax.legend()
ax.grid(alpha=0.3)

# Noise robustness
ax = axes[1]
for config in ["baseline", "t_c_e"]:
    if config in ood_results:
        noise_levels = [0.1, 0.2, 0.3, 0.5]
        accs = [ood_results[config]["noise"][f"noise_{s}"]["knn_accuracy"] for s in noise_levels]
        ax.plot(noise_levels, accs, marker='o', linewidth=2, label=config)

ax.set_xlabel("Noise σ")
ax.set_ylabel("kNN Accuracy")
ax.set_title("Noise Robustness")
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Few-Shot Learning Analysis

In [ ]:
# Compare few-shot performance across configurations
k_shots = [1, 3, 5, 10]
configs_to_compare = ["baseline", "t_only", "e_only", "t_c_e"]

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
methods = ["proto", "logreg", "knn"]
method_names = ["Prototypical", "Logistic Regression", "k-NN"]

for ax, method, name in zip(axes, methods, method_names):
    for config in configs_to_compare:
        if config in fewshot_results:
            means = [fewshot_results[config][f"{k}_shot"][f"{method}_mean"] for k in k_shots]
            stds = [fewshot_results[config][f"{k}_shot"][f"{method}_std"] for k in k_shots]
            ax.errorbar(k_shots, means, yerr=stds, marker='o', linewidth=2, capsize=5, label=config)
    
    ax.set_xlabel("k (shots per class)")
    ax.set_ylabel("Test Accuracy")
    ax.set_title(name)
    ax.legend()
    ax.grid(alpha=0.3)
    ax.set_ylim(0, 1)

plt.tight_layout()
plt.show()

## 6. Statistical Summary

In [ ]:
# Summary statistics
print("=" * 60)
print("OVERALL PERFORMANCE SUMMARY")
print("=" * 60)

if "baseline" in ablation_results and "t_c_e" in ablation_results:
    baseline = ablation_results["baseline"]["metrics"]
    full_aps = ablation_results["t_c_e"]["metrics"]
    
    print("\nAblation Study (t_c_e vs baseline):")
    for metric in ["trustworthiness", "continuity", "ari", "nmi", "silhouette"]:
        improvement = ((full_aps[metric] - baseline[metric]) / baseline[metric]) * 100
        print(f"  {metric:20s}: {baseline[metric]:.4f} → {full_aps[metric]:.4f} ({improvement:+.2f}%)")

print("\n" + "=" * 60)

## 7. Key Findings

### Topology Preservation (T)
- **Effect**: Improves trustworthiness and continuity metrics
- **Best for**: Maintaining local neighborhood structure

### Causality/Independence (C)
- **Effect**: Reduces spurious correlations
- **Best for**: OOD robustness

### Energy Basins (E)
- **Effect**: Creates clearer cluster structure
- **Best for**: Few-shot learning and clustering

### Full APS (T+C+E)
- **Synergy**: Components work together complementarily
- **Best overall**: Achieves highest scores across most metrics